In [1]:
!python -m pip install --upgrade pip
!python -m pip install -r requirements.txt


  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import scrapy

print("✅ All packages imported successfully!")


✅ All packages imported successfully!


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
# Define the URL and headers
url = "https://www.treasury.gov.lk"
headers = {
    'User-Agent': 'Group 5 DA2009 Project Scraper/1.0 (https://github.com/your_repo_name)'
}

# Add a User-Agent to your requests to mimic a browser.
# This is a key step in ethical scraping, as covered in Week 7[cite: 1158].

In [5]:
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Check for bad status codes (4xx or 5xx) [cite: 518, 532]
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Successfully fetched the page.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Successfully fetched the page.


In [6]:
# Assuming news headlines are inside <h2> tags with a specific class
news_items = soup.find_all('h2', class_='news-title')
scraped_data = []

# Loop through each item to extract the text and a link
for item in news_items:
    title = item.get_text(strip=True)
    link_tag = item.find('a')
    if link_tag:
        link = link_tag['href']
        scraped_data.append({'Title': title, 'Link': url + link})

# Convert to a DataFrame and display
df = pd.DataFrame(scraped_data)
df

""


In [7]:
# Try to scrape any HTML tables from the page
try:
    tables = pd.read_html(url)
    print(f"Found {len(tables)} tables on the page.")
    # The first table is usually at index 0. You can inspect others if needed.
    df_table = tables[0]
    print(df_table.head())
except Exception as e:
    print(f"No tables found or an error occurred: {e}")

Found 4 tables on the page.
  Currency     Buying    Selling
0      USD  297.08LKR  304.61LKR
1      GBP  397.64LKR  410.70LKR


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the driver (make sure the chromedriver is in your path or folder)
driver = webdriver.Chrome()

try:
    driver.get(url)

    # Wait for up to 10 seconds for the dynamic content (e.g., a table with the ID 'data-table') to be visible [cite: 1053]
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, 'data-table'))
    )

    # Now that the element has loaded, you can get the page source
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Scrape the dynamic content from the soup object
    dynamic_data_element = soup.find('table', id='data-table')

    # Now, you can process the dynamic_data_element with BeautifulSoup
    # ... your scraping logic here ...

finally:
    # Always remember to close the browser
    driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7bbc7e415+77285]
	GetHandleVerifier [0x0x7ff7bbc7e470+77376]
	(No symbol) [0x0x7ff7bba49a6a]
	(No symbol) [0x0x7ff7bbaa0406]
	(No symbol) [0x0x7ff7bbaa06bc]
	(No symbol) [0x0x7ff7bbaf3ac7]
	(No symbol) [0x0x7ff7bbac864f]
	(No symbol) [0x0x7ff7bbaf087f]
	(No symbol) [0x0x7ff7bbac83e3]
	(No symbol) [0x0x7ff7bba91521]
	(No symbol) [0x0x7ff7bba922b3]
	GetHandleVerifier [0x0x7ff7bbf61efd+3107021]
	GetHandleVerifier [0x0x7ff7bbf5c29d+3083373]
	GetHandleVerifier [0x0x7ff7bbf7bedd+3213485]
	GetHandleVerifier [0x0x7ff7bbc9884e+184862]
	GetHandleVerifier [0x0x7ff7bbca055f+216879]
	GetHandleVerifier [0x0x7ff7bbc87084+113236]
	GetHandleVerifier [0x0x7ff7bbc87239+113673]
	GetHandleVerifier [0x0x7ff7bbc6e298+11368]
	BaseThreadInitThunk [0x0x7ffa8df5e8d7+23]
	RtlUserThreadStart [0x0x7ffa8f69c34c+44]


In [10]:
# Save the scraped data to a CSV file
df.to_csv('data/scraped_treasury_data.csv', index=False)
print("Data saved successfully to data/scraped_treasury_data.csv")

Data saved successfully to data/scraped_treasury_data.csv


In [3]:
url = "https://www.treasury.gov.lk/"
tables = pd.read_html(url)  # returns list of DataFrames
df = tables[0]  # first table


In [4]:
import pandas as pd

url = "https://www.treasury.gov.lk/"
tables = pd.read_html(url)

print(f"Found {len(tables)} tables")

for i, table in enumerate(tables):
    print(f"\nTable {i} shape: {table.shape}")
    print(table.head())


Found 4 tables

Table 0 shape: (2, 3)
  Currency     Buying    Selling
0      USD  297.08LKR  304.61LKR
1      GBP  397.64LKR  410.70LKR

Table 1 shape: (3, 3)
                    Unnamed: 0   Year / Month Amount / LKR Bn
0  Governement Revenue & Grant  Jan-July 2024          2155.9
1       Government Expenditure  Jan-July 2024          3034.4
2       Overall Budget Deficit              .               .

Table 2 shape: (2, 4)
   #               Item   Pettah Dambulla
0  1              Samba  0LKR/Kg  0LKR/Kg
1  2  Red-Onions(Local)  0LKR/Kg  0LKR/Kg

Table 3 shape: (2, 4)
     Month/Year  Export  Import  Trade Balance
0  Jan-Nov 2023       0       0              0
1  Jan-Nov 2022       0       0              0


In [5]:
df = tables[2]  # replace with correct index
df.to_csv("treasury_data_raw.csv", index=False)


In [6]:
import os

folders = ["data_raw", "data_clean", "slides", "docs"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Project folders created:", folders)


Project folders created: ['data_raw', 'data_clean', 'slides', 'docs']


In [7]:
print("Test notebook")


Test notebook
